In [7]:
import requests
from bs4 import BeautifulSoup
from collections import deque
import json

In [80]:
url_base = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/consulta.php?IndiceFixo=&Conteudo=Disco&Procura=&FormularioAnterior=&Apartirde=0&NumeroItens=7325&Formulario=Exibir"

In [81]:
def get_links_discos(url_base):
    source = requests.get(url_base)
    soup = BeautifulSoup(source.content,  "html.parser")
    links_discos = ['http://www.discosdobrasil.com.br/discosdobrasil/consulta/' + \
                    a['href'] for a in soup.find_all('a', href=True) \
                    if 'Id_Disco' in a['href']]
    return list(set(links_discos))

In [48]:
def get_albumName(soup):
    album = soup.find('p', attrs={'class':"destacadorealcadorecuado"})
    return album.text.strip()

In [49]:
def get_interpretes(soup):
    interpretes = [a.text.strip() for a in soup.find('p', 
             attrs={'class': 'realcadorecuado'}).findChildren("a", recursive=True)]
    return interpretes

In [50]:
def get_infosDisco(soup):
    table = soup.find_all('table', attrs={'width': '460'})[2]
    
    chaves = [td.find("p", recursive=False).text.replace(':', '') 
              for td in table.find_all('td', attrs={'width': '25%'})]
    
    valores = [td.find("p", recursive=False).text.strip() 
           for td in table.find_all('td', attrs={'width': '75%'})]
    
    infos_disco = {chave:valor for chave, valor in zip(chaves, valores)}
    
    return infos_disco

In [51]:
def rangeFaixas(soup):
    tables = soup.find_all('table', {'width':"460",
                                 'border':"0",
                                 'align':"center",
                                 'cellpadding':"0",
                                 'cellspacing':"0",
                                 'bgcolor':"#FAF7D6"})
    indices = []
    for i in range(len(tables)):
        p = tables[i].find('p', {'class':"destacadorecuado"})
        if p != None and 'faixa' in p.text:
            indices.append(i)
        
    range_faixas = []
    n = indices[-1]
    for i in range(len(indices)):
        faixa = []
        if indices[i] < n:
            ind1, ind2 = indices[i], indices[i+1]
            faixa = tables[ind1:ind2]
            range_faixas.append(faixa)
        else:
            faixa = tables[n:]
            range_faixas.append(faixa)
    return range_faixas

In [52]:
def get_infoFaixa(range_faixa):
    musicos = []
    compositores = []
    musicas = []
    for table in range_faixa:
        a_ = table.find_all('a')
        p = table.find('p', {'class':"destacadorecuado"})
        for a in a_:
            if a != None and 'Id_Musica' in a['href']:
                musicas.append(a.text.strip())
            if a != None and 'Id_Musico' in a['href']:
                musico_instrumento = a.parent.text.strip()
                musicos.append(musico_instrumento)
            if a != None and 'Id_Compositor' in a['href']:
                compositor = a.text.strip()
                compositores.append(compositor)
        if p != None and 'faixa' in p.text:
            faixa = p.text.strip()
            
    dic = {'musica': musicas,
          'compositores': compositores,
          'musicos': musicos}
    return faixa, dic

In [82]:
links_discos = get_links_discos(url_base)

In [46]:
len(links_discos)

7325

In [47]:
links_discos

['http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04994',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00516',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00699',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI02508',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04648',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05575',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00434',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05525',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI06514',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00138',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI07102',
 'http://w

In [9]:
exe1 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05284"
ex2 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03054"
ex3 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03231"
ex4 = 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI07317'
exemplos = [exe1, ex2, ex3, ex4]

<table width="460" border="0" align="center" cellpadding="0" cellspacing="0" bgcolor="#FAF7D6">
        <tbody><tr>
            <td align="right" valign="center" width="20%">
                <p class="destacadorecuado">faixa 01</p>
            </td>
            <td valign="center" width="10%">
                <a href="../musicas/DI05284MU038761.WMA" onfocus="if(this.blur)this.blur()" onmouseout="MM_swapImgRestore()" onmouseover="MM_swapImage('ouvir','','../figuras/bt_ouvir_b.gif',1)"><img src="../figuras/bt_ouvir_a.gif" alt="Clique para ouvir a música" name="ouvir" width="30" height="30" border="0"></a>
            </td>
            <td valign="center" width="70%"><p class="realcadominirecuado"><a href="detalhe.php?Id_Musica=MU038761" onmouseover="ddrivetip(' Exibe a relação dos discos com esta música.')" ;="" onmouseout="hideddrivetip()" onfocus="if(this.blur)this.blur()" onclick="AbreJanela(this.href,'Detalhe83650','517','600','yes');return false;">O Atirador </a></p></td>
        </tr>
        <tr>
            <td valign="top" width="20%">
                &nbsp;
            </td>
            <td valign="top" width="10%">
                &nbsp;
            </td>
            <td valign="top" width="70%">
                &nbsp;
            </td>
        </tr>
    </tbody></table>

In [67]:
r = requests.get(exemplos[2])
soup = BeautifulSoup(r.content, 'html.parser')
r.close()

In [68]:
faixas = [p.text for p in soup.find_all('p', {'class':"destacadorecuado"}) if 'faixa' in p.text]
faixas

['faixa 01',
 'faixa 02',
 'faixa 03',
 'faixa 04',
 'faixa 05',
 'faixa 06',
 'faixa 07',
 'faixa 08',
 'faixa 09',
 'faixa 10',
 'faixa 11',
 'faixa 12']

In [69]:
faixas = rangeFaixas(soup)

In [70]:
range_faixa = faixas[10]

In [71]:
get_infoFaixa(range_faixa)

('faixa 11',
 {'musica': ['London, London'],
  'compositores': ['Caetano Veloso'],
  'musicos': ['Armandinho Macedo : Bandolim',
   'Ary Dias : Percussão',
   'Fernando Moura : Piano']})

In [72]:
get_albumName(soup)

'A VOZ DO BANDOLIM - CAETANO & GIL'

In [73]:
get_interpretes(soup)

['Armandinho Macedo']

In [74]:
get_infosDisco(soup)

{'Característica': 'instrumental',
 'Gravadora': 'Visom Digital',
 'Produtor': 'Carlão de Andrade/Ângela Nou',
 'Formatos': '(CD/2001)',
 'Primeiro disco': '2001',
 'Observação': '"O Bater Do Tambor" - na faixa 12 - está, erradamente, com o nome de "Toda Eletricidade" no disco.'}

# Interando e Salvando  

In [75]:
import json

def save_json(lista):
    with open('data.json', 'w', encoding='utf8') as file:
        json.dump(lista, file, ensure_ascii=False)

In [43]:
lista_dic = [{'Característica': 'instrumental',
 'Gravadora': 'Visom Digital',
 'Produtor': 'Carlão de Andrade/Ângela Nou',
 'Formatos': '(CD/2001)',
 'Primeiro disco': '2001',
 'Observação': '"O Bater Do Tambor" - na faixa 12 - está, erradamente, com o nome de "Toda Eletricidade" no disco.'}]

In [44]:
save_json(lista_dic)

In [57]:
def main(exemplos):
    albuns = []
    for link in exemplos:
        dic = dict()
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        get_infoFaixa(range_faixa)
        dic['album'] = get_albumName(soup)
        dic['interpretes'] = get_interpretes(soup)
        dic['info_album'] = get_infosDisco(soup)
        dic['faixas'] = dict()
        range_faixas = rangeFaixas(soup)
        for range_ in range_faixas:
            faixa, info_faixa = get_infoFaixa(range_)
            dic['faixas'][faixa] = info_faixa
            
        albuns.append(dic)
        save_json(albuns)

In [58]:
main(exemplos)

In [8]:
with open('data.json', 'r', encoding='utf8') as f:
    data = json.load(f)

In [9]:
len(data)

7325

In [63]:
print(json.dumps(data[0], indent=4, sort_keys=True, ensure_ascii=False))

{
    "album": "ACÚSTICO MTV - LENINE",
    "faixas": {
        "faixa 01": {
            "compositores": [
                "Lula Queiroga"
            ],
            "musica": [
                "O Atirador"
            ],
            "musicos": [
                "Guila : Baixo Semi-Acústico",
                "Guila : Voz",
                "Jr. Tostoi : Voz",
                "Jr. Tostoi : Craviola 12 Cordas",
                "Lenine : Violão",
                "Pantico Rocha : Bateria",
                "Pantico Rocha : Voz"
            ]
        },
        "faixa 02": {
            "compositores": [
                "Caxa Aragão",
                "Lenine",
                "Mu Chebadi"
            ],
            "musica": [
                "Hoje Eu Quero Sair Só"
            ],
            "musicos": [
                "Guila : Baixo Semi-Acústico",
                "Jr. Tostoi : Craviola 12 Cordas",
                "Lenine : Violão",
                "Pantico Rocha : Bateria"
            ]


In [65]:
print(json.dumps(data[2], indent=4, sort_keys=True, ensure_ascii=False))

{
    "album": "A VOZ DO BANDOLIM - CAETANO & GIL",
    "faixas": {
        "faixa 01": {
            "compositores": [
                "Gilberto Gil"
            ],
            "musica": [
                "Expresso 2222"
            ],
            "musicos": [
                "Armandinho Macedo : Bandolim",
                "Ary Dias : Percussão"
            ]
        },
        "faixa 02": {
            "compositores": [
                "Caetano Veloso"
            ],
            "musica": [
                "Trilhos Urbanos"
            ],
            "musicos": [
                "Armandinho Macedo : Bandolim",
                "Ary Dias : Percussão",
                "Dadi Carvalho : Baixo Elétrico",
                "Fernando Moura : Programação de Sampler",
                "Fernando Moura : Teclados",
                "Fernando Moura : Piano"
            ]
        },
        "faixa 03": {
            "compositores": [
                "Gilberto Gil"
            ],
            "musica": 

In [66]:
print(json.dumps(data[3], indent=4, sort_keys=True, ensure_ascii=False))

{
    "album": "7",
    "faixas": {
        "faixa 01": {
            "compositores": [
                "Rogério Caetano"
            ],
            "musica": [
                "Pingo De Ouro"
            ],
            "musicos": [
                "Gian Correa : Violão 7 Cordas Aço",
                "Rogério Caetano : Violão 7 Cordas Aço"
            ]
        },
        "faixa 02": {
            "compositores": [
                "Gian Correa"
            ],
            "musica": [
                "Rogerinho No Baile"
            ],
            "musicos": [
                "Gian Correa : Violão 7 Cordas Aço",
                "Rogério Caetano : Violão 7 Cordas Aço"
            ]
        },
        "faixa 03": {
            "compositores": [
                "Rogério Caetano"
            ],
            "musica": [
                "Agora É A Hora"
            ],
            "musicos": [
                "Gian Correa : Violão 7 Cordas Aço",
                "Rogério Caetano : Violão 7 Cordas

In [83]:
urls = deque(links_discos)

In [86]:
n = 0
for url in urls:
    print(url)
    n+=1
    if n == 5:
        break

http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03379
http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI01634
http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00954
http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00907
http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04764


In [88]:
import os

In [91]:
os.path.isfile('./urls.txt')

True

In [2]:
file = open('urls.txt', 'r')

In [3]:
urls = [url[:-1]for url in file.readlines()]

In [4]:
urls[:5]

['http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04701http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI01062http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03207http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03162http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI01597http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI06706http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03889http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04951http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI01526http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI02773http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00413http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalh

In [5]:
len(urls)

1